# Optimal clustering by Zemel et al. - Adult data (race)

This notebook contains an implementation of the pre-processing fairness intervention introduced in Learning Fair Representations by Zemel et al. (2013) as part of the IBM AIF360 fairness tool box github.com/IBM/AIF360. 

Here, we consider fairness defined with respect to race. There is another notebook considering fairness with respect to sex using Zemel et al.'s intervention method, which contains more details on the method. We follow analogous steps to the accompanying notebook addressing unfairness with respect to sex.

In [ ]:
from pathlib import Path

import joblib
import numpy as np
import pandas as pd
from aif360.algorithms.preprocessing.lfr import LFR  # noqa
from aif360.datasets import StandardDataset
from fairlearn.metrics import (
    demographic_parity_difference,
    demographic_parity_ratio,
)
from helpers.metrics import accuracy
from helpers.plot import group_bar_plots

## Load data

We have committed preprocessed data to the repository for reproducibility and we load it here. Check out hte preprocessing notebook for details on how this data was obtained.

In [ ]:
artifacts_dir = Path("../../../artifacts")

In [ ]:
data_dir = artifacts_dir / "data" / "adult"

train = pd.read_csv(data_dir / "processed" / "train-one-hot.csv")
val = pd.read_csv(data_dir / "processed" / "val-one-hot.csv")
test = pd.read_csv(data_dir / "processed" / "test-one-hot.csv")

### Remove points which are not white/black people

In order to analyse unfairness for binary protected attributes, namely, black / white race, we remove data points that correspond to races different to those two.

In [ ]:
for data in [train, test, val]:
    data = data[data.race_white + data.race_black == 1]
    data.drop(
        [
            "race_amer_indian_eskimo",
            "race_asian_pac_islander",
            "race_other",
            "race_black",
        ],
        axis=1,
    )

In order to process data for our fairness intervention we need to define special dataset objects which are part of every intervention pipeline within the IBM AIF360 toolbox. These objects contain the original data as well as some useful further information, e.g., which feature is the protected attribute as well as which column corresponds to the label.

In [ ]:
train_sds = StandardDataset(
    train,
    label_name="salary",
    favorable_classes=[1],
    protected_attribute_names=["race_white"],
    privileged_classes=[[1]],
)
test_sds = StandardDataset(
    test,
    label_name="salary",
    favorable_classes=[1],
    protected_attribute_names=["race_white"],
    privileged_classes=[[1]],
)
val_sds = StandardDataset(
    val,
    label_name="salary",
    favorable_classes=[1],
    protected_attribute_names=["race_white"],
    privileged_classes=[[1]],
)
index = train_sds.feature_names.index("race_white")

In [ ]:
privileged_groups = [{"race_white": 1.0}]
unprivileged_groups = [{"race_white": 0.0}]

## Demographic parity

Given the original unfair data set we apply Zemel et al.'s intervention to obtain a fair data set including fair labels. More precisely, we load an already learnt mitigation or learn a new mitigation procedure based on the true and predicted labels of the training data. We then apply the learnt procedure to transform the testing data and analyse fairness and accuracy in the transformed testing data.

The degree of fairness and accuracy can be controlled by the choice of parameters $A_x, A_y, A_z$ and $k$ when setting up the mitigation procedure. Here, $A_x$ controls the loss associated with the distance between original and transformed data set, $A_y$ the accuracy loss and $A_z$ the fairness loss. The larger one of these parameter is chosen compared to the others, the larger the priority of minimising the loss associated with that parameter. Hence, leaving $A_x$ and $A_y$ fixed, we can increase the degree of fairness achieved by increasing the parameter $A_z$.

As differences in fairness between independently learnt mitigations with same parameter choice can sometimes be significant we load a pre-trained intervention which achieves reasonable results. The user is still encouraged to train inteventions themselves (see commented out code below), and compare achieved fairness, potentially for a number of indepedent runs.

## Train unfair model

For maximum reproducibility we load the baseline model from disk, but the code used to train can be found in the baseline model notebook.

In [ ]:
bl_model = joblib.load(artifacts_dir / "models" / "finance" / "baseline.pkl")

bl_test_probs = bl_model.predict_proba(test_sds.features)[:, 1]
bl_test_pred = bl_test_probs > 0.5

## Load or learn intervention

So that you can reproduce our results we include a pretrained model, but the code for training your own model and experimenting with hyperparameters can be found below.

a) Location of the intervention previously learned on the training data.

In [ ]:
TR = joblib.load(artifacts_dir / "models" / "finance" / "zemel-race.pkl")

b) Learn intervention of the training data.

In [ ]:
# TR = LFR(
#     unprivileged_groups=unprivileged_groups,
#     privileged_groups=privileged_groups,
#     k=5,
#     Ax=0.01,
#     Ay=1.0,
#     Az=100.0,
# )
# TR = TR.fit(train_sds)

Apply intervention to test set.

In [ ]:
transf_test_sds = TR.transform(test_sds)
test_fair_labels = transf_test_sds.labels.flatten()

Evaluate fairness and accuracy on test data.

In [ ]:
bl_acc = bl_model.score(test.drop(columns="salary"), test.salary)
bl_dpd = demographic_parity_difference(
    test.salary, bl_test_pred, sensitive_features=test.race_white,
)
bl_dpr = demographic_parity_ratio(
    test.salary, bl_test_pred, sensitive_features=test.race_white,
)

acc = accuracy(test.salary, test_fair_labels)
dpd = demographic_parity_difference(
    test.salary, test_fair_labels, sensitive_features=test.race_white,
)
dpr = demographic_parity_ratio(
    test.salary, test_fair_labels, sensitive_features=test.race_white,
)

print(f"Baseline accuracy: {bl_acc:.3f}")
print(f"Accuracy: {acc:.3f}\n")

print(f"Baseline demographic parity difference: {bl_dpd:.3f}")
print(f"Demographic parity difference: {dpd:.3f}\n")

print(f"Baseline demographic parity ratio: {bl_dpr:.3f}")
print(f"Demographic parity ratio: {dpr:.3f}")

In [ ]:
dp_bar = group_bar_plots(
    np.concatenate([bl_test_pred, test_fair_labels]),
    np.tile(test.race_white.map({0: "Black", 1: "White"}), 2),
    groups=np.concatenate(
        [np.zeros_like(bl_test_pred), np.ones_like(test_fair_labels)]
    ),
    group_names=["Baseline", "Zemel"],
    title="Proportion of predicted high earners by race",
    xlabel="Propotion of predicted high earners",
    ylabel="Method",
)
dp_bar